In [0]:
import pandas as pd #데이터 전처리
import numpy as np #데이터 전처리
import pandas_profiling #데이터 전처리

import matplotlib.pyplot as plt #데이터 시각화
import seaborn as sns #데이터 시각화
sns.set_style('whitegrid')# seaborn 패키지의 배경 지정

## Data Load

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./submission_1002.csv')
print('train shape: ', train.shape)
print('test shape: ', test.shape)
print('submission shape: ', submission.shape)

train shape:  (15789, 1301)
test shape:  (8760, 201)
submission shape:  (200, 40)


## Conver time type tp datetime

In [0]:
train['Time'] = pd.to_datetime(train.Time) #날짜 형식으로 변환 작업을 합니다.
train = train.set_index('Time') #'Time'열을 기본 index로 설정합니다.

test['Time'] = pd.to_datetime(test.Time)
test = test.set_index('Time')

## Missning Values

In [4]:
train.head(3)

,X692,X1272,X553,X1299,X4,X598,X1003,X1010,X1216,X1047,X381,X466,X1420,X686,X711,X1251,X828,X1350,X1066,X1293,X1148,X451,X174,X74,X626,X1083,X1000,X632,X1380,X607,X680,X977,X492,X968,X891,X122,X489,X86,X392,X1007,...,X109,X224,X1259,X1143,X1324,X889,X689,X447,X1270,X827,X1037,X942,X1498,X678,X1100,X239,X1369,X436,X536,X933,X1035,X1194,X1441,X291,X1328,X1323,X616,X1195,X944,X1017,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-07-26 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
_, ax = plt.subplots(1,2, figsize=(15,5)) #train, test를 한 번에 비교하기 위해, 그래프 창을 2개로 만듭니다.
# train.isnull().mean(axis=0) #각 세대별 이름과 결측치 비율이 나열됩니다.
sns.distplot(train.isnull().mean(axis=0), ax=ax[0]) #나열된 값을 distplot을 이용해 시각화 하고, 이를 첫 번째 그래프 창에 넣습니다.
ax[0].set_title('Distribution of Missing Values Percentage in Train set')

sns.distplot(test.isnull().mean(axis=0), ax=ax[1]) #test data에서의 결측치 비율을 시각화 하고, 이를 두 번째 그래프 창에 넣습니다.
ax[1].set_title('Distribution of Missing Values Percentage in Test set')
plt.show()

## Average Score

In [0]:
answer = test.median(axis=0).sort_index() #각 세대 별 중앙에 위치한 값인 중앙값을 계산합니다.

avg_submission = submission.copy() #원본 데이터 보존을 위한 데이터 복사
avg_submission = avg_submission.set_index('meter_id') #'meter_id'를 기본 index로 설정합니다.

for i in range(24):
    avg_submission.iloc[:,i] = answer #각 세대, 시간별 예측 값을 넣습니다. #시간별 예측 값은 중앙값으로 동일합니다
    
for i in range(10):
    avg_submission.iloc[:,24+i] = answer * 24 #각 세대, 일자별 예측값을 넣습니다.  
    
#각 세대 월 별 계산하기
months = np.zeros((answer.shape[0], 5)) # 200 * 5 로 이루어진 0 값을 사전에 만들어 놓습니다.
months[:,0] = answer * 24 * 31 # 시간별 예측값에 24를 곱해 일자별 예측값으로 만들고 31일간의 값을 구하기 위해, 31을 곱합니다.
months[:,1] = answer * 24 * 31 # 동일하게 8월을 예측하기 위해, 31을 곱합니다.
months[:,2] = answer * 24 * 30 # 동일하게 9월을 예측하기 위해, 30을 곱합니다.
months[:,3] = answer * 24 * 31 # 동일하게 10월을 예측하기 위해, 31을 곱합니다.
months[:,4] = answer * 24 * 30 # 동일하게 11월을 예측하기 위해, 30을 곱합니다.

avg_submission.iloc[:,34:] = months #미리 만들어둔 각 세대, 월별 예측값을 넣습니다.